In [ ]:
# Importação de bibliotecas
import pandas as pd

from pendulum import now

### Tabela 1613 - Área destinada à colheita, área colhida, quantidade produzida, rendimento médio e valor da produção das lavouras permanentes 

In [ ]:
fonte = 'sidra'
dataset_name = 'tab1613_lavoura_permanente'
raiz = '/home/eduardo/repositorios/IS_Agro/databases'

ultimo_ano = now().year - 1

#### Bronze

Área destinada à colheita (ha)

In [ ]:
culturas = [2717, 2718, 2719, 2720, 2721, 2722, 2723, 2724, 2725, 2726, 2727, 2728, 2729, 2730, 2731, 2732, 2733, 2734, 2735, 2736, 2737,
            2738, 2739, 2740, 2741, 2742, 2743, 2744, 2745, 2746, 2747, 2748, 45981, 90001]

variavel = 'area_destinada_colheita'

contador = 1

In [ ]:
for ano in range(2003, ultimo_ano):
    for cultura in culturas:
        if contador == 1:
            df_1613 = pd.read_json(f'https://apisidra.ibge.gov.br/values/t/1613/n6/all/v/2313/p/{ano}/c82/{cultura}')
            contador += 1
        else:
            df_1613 = pd.concat(
                [df_1613,
                 (pd.read_json(f'https://apisidra.ibge.gov.br/values/t/1613/n6/all/v/2313/p/{ano}/c82/{cultura}')
                  .iloc[1:])],
                  ignore_index=True
                  )
            contador += 1

df_1613.to_json(f'{raiz}/bronze_{fonte}_{dataset_name}_{variavel}.json')

print(f'Foram contabilizadas {contador} repetições de dados de áreas destinadas à colheita raspados.')

Área colhida

In [ ]:
culturas = [2717, 2720, 2723, 2724, 2725, 2730, 2731, 2733, 2734, 2735, 2736, 2738, 2739, 2741, 2742, 2745, 2748, 31619, 31620, 45981]

variavel = 'area_colhida'

contador = 1

In [ ]:
for ano in range(1990, ultimo_ano):
    for cultura in culturas:
        if contador == 1:
            df_1613ac = pd.read_json(f'https://apisidra.ibge.gov.br/values/t/1613/n6/all/v/216/p/{ano}/c82/{cultura}')
            contador += 1
        else:
            df_1613ac = pd.concat(
                [df_1613ac,
                 (pd.read_json(f'https://apisidra.ibge.gov.br/values/t/1613/n6/all/v/216/p/{ano}/c82/{cultura}')
                  .iloc[1:])],
                  ignore_index=True
                  )
            contador += 1

df_1613ac.to_json(f'{raiz}/bronze_{fonte}_{dataset_name}_{variavel}.json')

print(f'Foram contabilizadas {contador} repetições de dados de áreas colhidas raspados.')

#### Prata

| Código | Cultura                        |
|--------|--------------------------------|
| 2717   | Abacate                        |
| 2718   | Algodão arbóreo (em caroço)   |
| 2719   | Azeitona                       |
| 2720   | Banana (cacho)                 |
| 2721   | Borracha (látex coagulado)     |
| 2722   | Cacau (em amêndoa)             |
| 2723   | Café (em grão) Total           |
| 2724   | Caqui                          |
| 2725   | Castanha de caju               |
| 2726   | Chá-da-índia (folha verde)     |
| 2727   | Coco-da-baía*                  |
| 2728   | Dendê (cacho de coco)          |
| 2729   | Erva-mate (folha verde)        |
| 2730   | Figo                           |
| 2731   | Goiaba                         |
| 2732   | Guaraná (semente)              |
| 2733   | Laranja                        |
| 2734   | Limão                          |
| 2735   | Maçã                           |
| 2736   | Mamão                          |
| 2737   | Manga                          |
| 2738   | Maracujá                       |
| 2739   | Marmelo                        |
| 2740   | Noz (fruto seco)               |
| 2741   | Pera                           |
| 2742   | Pêssego                        |
| 2743   | Pimenta-do-reino               |
| 2744   | Sisal ou agave (fibra)         |
| 2745   | Tangerina                      |
| 2746   | Tungue (fruto seco)            |
| 2747   | Urucum (semente)               |
| 2748   | Uva                            |
| 45981  | Açaí                           |
| 90001  | Palmito                        |

In [ ]:
# Área destinada à colheita (área utilizada)
df_1613 = (
    pd.read_json(f'{raiz}/bronze_{fonte}_{dataset_name}_{variavel}.json')
        .drop(0, axis = 0)
        .filter(['D1C', 'D3C', 'D4C', 'V'])
        .assign(**{
            'area': lambda df: pd.to_numeric(df['V'], errors='coerce'),
            'geocodigo': lambda df: df['D1C'].astype(int),
            'data': lambda df: pd.to_datetime(df['D3C'], format='%Y'),
            'cultura': lambda df: df['D4C'].astype(int)
            })
        .drop(['D1C', 'D3C', 'D4C', 'V'], axis='columns')
        .dropna(subset='area')
        .set_index(['geocodigo', 'data', 'cultura'])
    )

df_1613.to_parquet(f'{raiz}/prata_{fonte}_{dataset_name}_{variavel}.parquet')

In [ ]:
# Área colhida
df_1613ac = (
    pd.read_json(f'{raiz}/bronze_{fonte}_{dataset_name}_{variavel}.json')
        .drop(0, axis = 0)
        .filter(['D1C', 'D3C', 'D4C', 'V'])
        .assign(**{
            'area': lambda df: pd.to_numeric(df['V'], errors='coerce'),
            'geocodigo': lambda df: df['D1C'].astype(int),
            'data': lambda df: pd.to_datetime(df['D3C'], format='%Y'),
            'cultura': lambda df: df['D4C'].astype(int)
            })
        .drop(['D1C', 'D3C', 'D4C', 'V'], axis='columns')
        .dropna(subset='area')
        .set_index(['geocodigo', 'data', 'cultura'])
)
df_1613ac.to_parquet(f'{raiz}/prata_{fonte}_{dataset_name}_{variavel}.parquet')

### Ouro

In [ ]:
dataset_ouro = 'ouro_volume_agricola_lavouras'

In [ ]:
df_1613_final = pd.read_parquet(f'{raiz}/prata_{fonte}_{dataset_name}_{variavel}.parquet')

#df_1613_final.to_csv(f'{raiz}/{dataset_ouro}_permanentes.csv')

In [ ]:
# Exportação para uso no indicador de Área Agropecuária
(
    df_1613_final
    .reset_index()
    .assign(geocodigo_uf = lambda df: df.geocodigo // 100000)
    .groupby(['geocodigo_uf', 'data', 'cultura'])
    .agg({'area':'sum'})
    .rename_axis(index={'geocodigo_uf': 'geocodigo'})
).to_csv(f'{raiz}/{dataset_ouro}_perenes.csv')

In [ ]:
df_1613_consolidado_munic = (df_1613_final.reset_index()
                             .groupby(['geocodigo', 'data'])
                             .agg({'area':sum})
                             )
df_1613_consolidado_munic.to_csv(f'{raiz}/{dataset_ouro}_permanentes_munic.csv')

In [ ]:
df_1613_consolidado_uf = (df_1613_final.reset_index()
                          .assign(geocodigo_uf = lambda df: df.geocodigo // 100000)
                          .groupby(['geocodigo_uf', 'data'])
                          .agg({'area':sum})
                          .rename_axis(index={'geocodigo_uf': 'geocodigo'})
                          )
df_1613_consolidado_uf.to_csv(f'{raiz}/{dataset_ouro}_permanentes_uf.csv')